# Pseudochannel Explorer

Interactive tool for creating weighted pseudochannel composites from multiplex tissue imaging data.

## Supported Input Formats
- **Folder of TIFFs**: Individual channel images in a folder (channel names from filenames)
- **OME-TIFF**: Single multi-channel file with marker names from a separate file

## Workflow
1. Configure paths in Section 1
2. Load channels (auto-detects format)
3. Interactively tune weights using sliders
4. Save configuration for reuse
5. Apply to full-resolution images or batch process

## 1. Setup & Configuration

In [ ]:
import sys
from pathlib import Path

# Add src to path if running from notebooks folder
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from pseudochannel import (
    DEFAULT_EXCLUDED_CHANNELS,
    load_channel_folder,
    load_ome_tiff,
    load_marker_names,
    load_mcmicro_markers,
    OMETiffChannels,
    compute_pseudochannel,
    create_preview_stack,
    save_config,
    load_config,
    process_dataset,
)
from pseudochannel.widgets import create_interactive_explorer, PseudochannelExplorer
from pseudochannel.batch import (
    batch_process_directory,
    find_channel_folders,
    process_ome_tiff,
    process_ome_tiff_batch,
)
from pseudochannel.config import get_weights_from_config, list_configs

import numpy as np
import matplotlib.pyplot as plt
import tifffile

# Use widget backend for interactive zoom feature
%matplotlib widget

In [ ]:
# =============================================================================
# CONFIGURATION - Set your paths here
# =============================================================================

# --- Input Data ---
# Set the path(s) for your data. The system will auto-detect which format to use.
# - If only CHANNEL_FOLDER is set: loads from folder of TIFFs
# - If only OME_TIFF_PATH is set: loads from OME-TIFF
# - If both are set: uses PREFER_OME_TIFF flag to decide

# Option A: Folder with individual channel TIFFs
# CHANNEL_FOLDER = None  # e.g., "/path/to/channel_folder"
CHANNEL_FOLDER = "/mnt/Vol_c/Seb_carcinoma/data/ROI13"  # e.g., "/path/to/channel_folder"

# Option B: OME-TIFF with marker file
OME_TIFF_PATH = None   # e.g., "/path/to/sample.ome.tiff"
MARKER_FILE = None     # e.g., "/path/to/markers.csv"
MARKER_COLUMN = None   # Column name/index for CSV files, or None for plain text
MCMICRO_MARKERS = False  # Set True if using MCMICRO markers.csv format (filters remove=TRUE)

# If both folder and OME-TIFF are provided, which to prefer?
PREFER_OME_TIFF = False  # False = prefer folder (Option A), True = prefer OME-TIFF (Option B)

# --- Channel Exclusions ---
# None = use defaults (DAPI, None, APC, FE, Autofluorescence, etc.)
# [] = include ALL channels
# ["DAPI", "Empty"] = exclude specific channels
EXCLUDE_CHANNELS = None

# --- Nuclear Marker (for overlay feature) ---
# For folder input: path to DAPI/nuclear stain TIFF file
# For OME-TIFF: leave as None to auto-detect DAPI channel
NUCLEAR_MARKER_PATH = None  # e.g., "/path/to/DAPI.tif"

# --- Output Paths ---
CONFIG_DIR = "../configs"
OUTPUT_DIR = "../outputs"

# =============================================================================
# Show configuration summary
print("Configuration:")
print(f"  Channel folder: {CHANNEL_FOLDER}")
print(f"  OME-TIFF path:  {OME_TIFF_PATH}")
print(f"  Marker file:    {MARKER_FILE}")
print(f"  MCMICRO format: {MCMICRO_MARKERS}")
print(f"  Prefer OME-TIFF: {PREFER_OME_TIFF}")
print(f"\nDefault excluded channels: {sorted(DEFAULT_EXCLUDED_CHANNELS)}")

## 2. Load Channels

The cell below automatically detects and loads from the appropriate source.

In [ ]:
def detect_input_mode(channel_folder, ome_tiff_path, marker_file, prefer_ome_tiff=False):
    """Detect which input mode to use based on provided paths.
    
    Returns:
        str: 'folder', 'ome_tiff', or raises ValueError if no valid input
    """
    has_folder = channel_folder is not None and Path(channel_folder).is_dir()
    has_ome_tiff = (
        ome_tiff_path is not None 
        and Path(ome_tiff_path).is_file() 
        and marker_file is not None 
        and Path(marker_file).is_file()
    )
    
    if has_folder and has_ome_tiff:
        mode = 'ome_tiff' if prefer_ome_tiff else 'folder'
        print(f"Both inputs available. Using {'OME-TIFF' if prefer_ome_tiff else 'folder'} (PREFER_OME_TIFF={prefer_ome_tiff})")
        return mode
    elif has_folder:
        print("Detected: Folder of TIFFs")
        return 'folder'
    elif has_ome_tiff:
        print("Detected: OME-TIFF with marker file")
        return 'ome_tiff'
    elif ome_tiff_path is not None and marker_file is None:
        raise ValueError("OME-TIFF path provided but MARKER_FILE is missing")
    else:
        raise ValueError(
            "No valid input detected. Please set either:\n"
            "  - CHANNEL_FOLDER (path to folder with TIFF files)\n"
            "  - OME_TIFF_PATH and MARKER_FILE (for OME-TIFF input)"
        )

# Detect input mode
INPUT_MODE = detect_input_mode(CHANNEL_FOLDER, OME_TIFF_PATH, MARKER_FILE, PREFER_OME_TIFF)

In [ ]:
# Load channels based on detected mode
if INPUT_MODE == 'folder':
    channels = load_channel_folder(CHANNEL_FOLDER, exclude_channels=EXCLUDE_CHANNELS)
    print(f"Loaded {len(channels)} channels from folder:")
    
elif INPUT_MODE == 'ome_tiff':
    # Show marker file contents first
    if MCMICRO_MARKERS:
        marker_names = load_mcmicro_markers(MARKER_FILE)
        print(f"Loaded {len(marker_names)} markers from MCMICRO file (after filtering):")
    else:
        marker_names = load_marker_names(MARKER_FILE, column=MARKER_COLUMN)
        print(f"Loaded {len(marker_names)} markers from file:")
    
    for i, name in enumerate(marker_names[:10]):
        print(f"  {i}: {name}")
    if len(marker_names) > 10:
        print(f"  ... and {len(marker_names) - 10} more")
    print()
    
    # Load OME-TIFF using lazy loading for large files
    channels = OMETiffChannels(
        OME_TIFF_PATH,
        MARKER_FILE,
        marker_column=MARKER_COLUMN,
        exclude_channels=EXCLUDE_CHANNELS,
        mcmicro_markers=MCMICRO_MARKERS,
    )
    print(f"Loaded {len(channels)} channels from OME-TIFF:")

# Show loaded channels
channel_list = list(channels.keys())
for name in channel_list[:10]:
    print(f"  {name}")
if len(channel_list) > 10:
    print(f"  ... and {len(channel_list) - 10} more")

## 3. Interactive Weight Tuning

Use the sliders to adjust the weight of each channel. The preview updates in real-time.

**Tips:**
- Draw a rectangle on the preview to zoom in at full resolution
- Enable "Show Nuclear (DAPI)" to overlay nuclear marker in blue
- Use the Columns/Layout dropdowns to adjust the interface for your screen

In [ ]:
# Create interactive explorer (works for both input modes)
if INPUT_MODE == 'folder':
    explorer = create_interactive_explorer(
        CHANNEL_FOLDER,
        preview_size=512,
        exclude_channels=EXCLUDE_CHANNELS,
        nuclear_marker_path=NUCLEAR_MARKER_PATH,
    )
elif INPUT_MODE == 'ome_tiff':
    explorer = create_interactive_explorer(
        OME_TIFF_PATH,
        marker_file=MARKER_FILE,
        marker_column=MARKER_COLUMN,
        preview_size=512,
        exclude_channels=EXCLUDE_CHANNELS,
        nuclear_marker_path=NUCLEAR_MARKER_PATH,
        mcmicro_markers=MCMICRO_MARKERS,
    )

In [ ]:
# Get current weights from sliders
current_weights = explorer.get_weights()
active_weights = {k: v for k, v in current_weights.items() if v > 0}

print(f"Active weights ({len(active_weights)} channels):")
for name, weight in sorted(active_weights.items(), key=lambda x: -x[1]):
    print(f"  {name}: {weight:.2f}")

## 4. Apply to Full Resolution

Once satisfied with weights, compute the full-resolution pseudochannel.

In [ ]:
# Compute full-resolution pseudochannel
weights = explorer.get_weights()

print("Computing full-resolution pseudochannel...")
full_res_result = compute_pseudochannel(
    channels,
    weights,
    normalize="minmax"
)
print(f"Result shape: {full_res_result.shape}")

In [ ]:
# Display full-resolution result (downsampled for display)
from pseudochannel.preview import downsample_image

display_preview = downsample_image(full_res_result, target_size=1024)

fig, ax = plt.subplots(figsize=(10, 10))
fig.canvas.header_visible = False
ax.imshow(display_preview, cmap='gray')
ax.set_title('Full Resolution Pseudochannel (downsampled for display)')
ax.axis('off')
plt.show()

In [ ]:
# Save full-resolution result as TIFF
output_path = Path(OUTPUT_DIR) / "pseudochannel_output.tif"
output_path.parent.mkdir(parents=True, exist_ok=True)

# Convert to 16-bit for saving
output_16bit = (full_res_result * 65535).astype(np.uint16)
tifffile.imwrite(str(output_path), output_16bit)

print(f"Saved to: {output_path}")

## 5. Save Configuration

Save the weight configuration for reuse with other images.

In [ ]:
# Save configuration
config_name = "my_pseudochannel_config"  # Change this name as needed
config_description = "Custom pseudochannel weights for membrane visualization"

config_path = save_config(
    weights=explorer.get_weights(),
    output_path=Path(CONFIG_DIR) / f"{config_name}.yaml",
    name=config_name,
    description=config_description,
    normalization="minmax"
)

print(f"Configuration saved to: {config_path}")

In [ ]:
# List available configurations
configs = list_configs(CONFIG_DIR)

if configs:
    print("Available configurations:")
    for cfg in configs:
        print(f"  {cfg['name']}: {cfg['num_channels']} channels - {cfg['description'][:50]}")
else:
    print("No saved configurations found.")

In [ ]:
# Load a saved configuration
config_to_load = "my_pseudochannel_config"  # Change to your config name

config = load_config(Path(CONFIG_DIR) / f"{config_to_load}.yaml")
explorer.set_weights(get_weights_from_config(config))
print(f"Loaded configuration: {config_to_load}")

## 6. Batch Processing

Apply the saved configuration to multiple images.

In [ ]:
# Find all channel folders in a directory (for folder-based input)
data_root = Path("../data")
folders = find_channel_folders(data_root)
print(f"Found {len(folders)} channel folders:")
for f in folders[:5]:
    print(f"  {f}")
if len(folders) > 5:
    print(f"  ... and {len(folders) - 5} more")

In [ ]:
# Batch process all folders using a saved config
config_to_use = "my_pseudochannel_config"

output_paths = batch_process_directory(
    root_path="../data",
    config_path=Path(CONFIG_DIR) / f"{config_to_use}.yaml",
    output_folder="../outputs/batch",
)

print(f"Processed {len(output_paths)} folders")

In [ ]:
# Batch process OME-TIFF files with a shared marker file
tiff_files = [
    "../data/sample1.ome.tiff",
    "../data/sample2.ome.tiff",
]
shared_marker_file = "../data/markers.csv"
config_to_use = "my_pseudochannel_config"

output_paths = process_ome_tiff_batch(
    tiff_files=tiff_files,
    marker_files=shared_marker_file,
    config_path=Path(CONFIG_DIR) / f"{config_to_use}.yaml",
    output_folder="../outputs/batch",
)

print(f"Processed {len(output_paths)} OME-TIFF files")

## Reference: Marker File Formats

### Plain Text (one marker per line)
```
DAPI
CD45
E-cadherin
Pan-CK
```

### CSV with header
```csv
channel_id,marker_name,target
1,DAPI,Nucleus
2,CD45,Immune
3,E-cadherin,Epithelial
```
Use: `MARKER_COLUMN = "marker_name"`

### MCMICRO format
```csv
channel_number,cycle_number,marker_name,Filter,background,exposure,remove
1,1,DAPI,DAPI,100,10,TRUE
2,1,CD45,Cy5,50,100,FALSE
```
Use: `MCMICRO_MARKERS = True` (automatically reads `marker_name` column and filters `remove=TRUE`)